In [31]:
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import awkward as ak
import matplotlib.pyplot as plt
import numba
from numba import njit
import numpy as np
from pocket_coffea.lib.parton_provenance import reverse_index_array

# use the last b quark for the matching

In [32]:
j=0
filename ="root://storage01.lcg.cscs.ch:1096//pnfs/lcg.cscs.ch/cms/trivcat//store/mc/Run3Summer22EENanoAODv12/GluGlutoHHto4B_kl-1p00_kt-1p00_c2-0p00_TuneCP5_13p6TeV_powheg-pythia8/NANOAODSIM/Poisson60KeepRAW_130X_mcRun3_2022_realistic_postEE_v6-v2/50000/fc603037-ef65-4bbf-9cef-934ecec40bbe.root"
filename ="/pnfs/psi.ch/cms/trivcat/store/user/mmalucch/hh4b_12.root"
events = NanoEventsFactory.from_root(filename, schemaclass=NanoAODSchema, entry_start=j, entry_stop=j+100000).events()
print("Events read:", len(events))

Events read: 57232


In [33]:
# add column with the index
events.GenPart = ak.with_field(
    events.GenPart, ak.local_index(events.GenPart, axis=1), "index"
)
print(events.GenPart.index[0])
print(events.GenPart.index[1])
# get last bquark copy
isB = abs(events.GenPart.pdgId) == 5
isLast = events.GenPart.hasFlags(["isLastCopy"])
isHard = events.GenPart.hasFlags(["fromHardProcess"])
bquarks_last = events.GenPart[isB & isLast & isHard]

print(bquarks_last.pdgId, bquarks_last.genPartIdxMother, bquarks_last.childrenIdxG, bquarks_last.hasFlags(["isLastCopy"]))
bquark_last_children=bquarks_last.children
print(bquark_last_children.pdgId, bquark_last_children.genPartIdxMother, bquark_last_children.hasFlags(["isLastCopy"]))

b_quarks = bquarks_last
while True:
    print("\nloop")
    b_mother = events.GenPart[b_quarks.genPartIdxMother]
    mask_mother = (abs(b_mother.pdgId) == 5) | ((b_mother.pdgId) == 25)

    bquarks_last = bquarks_last[mask_mother]
    b_quarks = b_quarks[mask_mother]
    b_mother = b_mother[mask_mother]
    print(
        "old: ",
        "pdg",
        b_quarks.pdgId,
        "mother_idx",
        b_quarks.genPartIdxMother,
        "pt",
        b_quarks.pt,
        "index",
        b_quarks.index,
        "fromHardProcess",
        b_quarks.hasFlags(["fromHardProcess"]),
    )
    print(
        "mother: ",
        "pdg",
        b_mother.pdgId,
        "mother_idx",
        b_mother.genPartIdxMother,
        "pt",
        b_mother.pt,
    )
    # break
    b_quarks = ak.where(abs(b_mother.pdgId) == 5, b_mother, b_quarks)
    print(
        "new: ",
        "pdg",
        b_quarks.pdgId,
        "mother_idx",
        b_quarks.genPartIdxMother,
        "pt",
        b_quarks.pt,
    )

    print(ak.any(abs(b_mother.pdgId) != 25, axis=1))
    if ak.all(abs(b_mother.pdgId) == 25):
        break

print(b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(b_quarks.genPartIdxMother, len(b_quarks.genPartIdxMother))
print(b_quarks.index, len(b_quarks.index))
print(bquarks_last.pdgId, bquarks_last.index,  bquarks_last.pt, len(bquarks_last.index), bquarks_last.hasFlags(["fromHardProcess"]))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, ... 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65]
[[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [-5, 5, 5, -5], [5, 5, -5, -5]] [[17, 18, 19, 20], [17, 16, 20, 19], [20, ... [26, 25, 23, 24], [18, 16, 17, 19]] [[[38, 39, 59, 60], [], [], []], [[], [], [], ... [[], [], [], []], [[], [], [], []]] [[True, True, True, True], [True, True, ... True, True], [True, True, True, True]]
[[[-513, 523, 11, -11], [], [], []], [[], [], ... [], [], [], []], [[], [], [], []]] [[[33, 33, 33, 33], [], [], []], [[], [], [], ... [[], [], [], []], [[], [], [], []]] [[[True, True, True, True], [], [], []], [[], ... [], [], []], [[], [], [], []]]

loop
old:  pdg [[5, -5, 5, -5], [-5, 5, 5, -5], [-5, 5, ... 5, -5], [-5, 5, 5, -5], [5, 5, -5, -5]] mother_idx [[17, 18, 19, 20], [17, 16, 20, 19], [20, ... [26, 25, 23, 24], [18, 16, 17, 19]] pt [[116, 124, 186, 13.8], [87.2, 35.4, 103, ... 27.

In [34]:
print(bquarks_last.index, len(bquarks_last.index))


[[33, 37, 40, 42], [21, 27, 32, 33], [21, ... [27, 33, 38, 40], [20, 22, 37, 40]] 57232


# NUMBA

In [35]:
genpart =events.GenPart

# get last bquark copy
isB = abs(genpart.pdgId) == 5
isLast = genpart.hasFlags(["isLastCopy"])
isFirst = genpart.hasFlags(["isFirstCopy"])
isHard = genpart.hasFlags(["fromHardProcess"])

isHiggs = genpart.pdgId == 25
higgs = genpart[isHiggs & isLast & isHard]
higgs = higgs[ak.num(higgs.childrenIdxG, axis=2) == 2]

higgs = higgs[ak.argsort(higgs.pt, ascending=False)]


In [36]:
# bquarks = genpart[isB & isHard & isFirst]
bquarks = genpart[isB & isHard]
mother_bquarks = genpart[bquarks.genPartIdxMother]
print(bquarks.index, mother_bquarks.index,mother_bquarks.childrenIdxG, mother_bquarks.pdgId)
print(mother_bquarks.pdgId == 25)
bquarks_from_higgs = bquarks[mother_bquarks.pdgId == 25]

print(bquarks_from_higgs.index, bquarks_from_higgs.genPartIdxMother)
print("bquarks_from_higgs", bquarks_from_higgs.index, bquarks_from_higgs.pdgId)

[[17, 18, 19, 20, 33, 37, 40, 42], [16, ... 40], [16, 17, 18, 19, 20, 22, 37, 40]] [[15, 15, 16, 16, 17, 18, 19, 20], [14, ... 24], [14, 14, 15, 15, 18, 16, 17, 19]] [[[17, 18], [17, 18], [19, 20, ... 3692853, 3692868], [3692866, 3692867], [3692870]]] [[25, 25, 25, 25, 5, -5, 5, -5], [25, 25, ... 5, -5], [25, 25, 25, 25, 5, 5, -5, -5]]
[[True, True, True, True, False, False, ... True, True, False, False, False, False]]
[[17, 18, 19, 20], [16, 17, 18, 19], [17, ... [23, 24, 25, 26], [16, 17, 18, 19]] [[15, 15, 16, 16], [14, 14, 15, 15], [15, ... [20, 20, 21, 21], [14, 14, 15, 15]]
bquarks_from_higgs [[17, 18, 19, 20], [16, 17, 18, 19], [17, ... [23, 24, 25, 26], [16, 17, 18, 19]] [[5, -5, 5, -5], [5, -5, 5, -5], [5, -5, ... 5, -5], [5, -5, 5, -5], [5, -5, 5, -5]]


In [48]:
# check that bquarks_from_higgs are -5 and 5
ak.any(ak.sum(higgs.children.pdgId, axis=2)!=0)


False

In [37]:
children_idxG = ak.without_parameters(genpart.childrenIdxG, behavior={})
children_idxG_flat = ak.flatten(children_idxG, axis=1)
genpart_pdgId_flat = ak.flatten(
    ak.without_parameters(genpart.pdgId, behavior={}), axis=1
)
genpart_LastCopy_flat = ak.flatten(
    ak.without_parameters(genpart.hasFlags(["isLastCopy"]), behavior={}), axis=1
)
genpart_pt_flat = ak.flatten(ak.without_parameters(genpart.pt, behavior={}), axis=1)
genparts_flat = ak.flatten(genpart)
genpart_offsets = np.concatenate(
    [[0], np.cumsum(ak.to_numpy(ak.num(genpart, axis=1), allow_missing=True))]
)
local_index_all = ak.local_index(genpart, axis=1)
local_index_b = ak.local_index(bquarks_from_higgs, axis=1)
b_quark_idx = ak.to_numpy(bquarks_from_higgs.index + genpart_offsets[:-1], allow_missing=False)
b_quarks_pdgId = ak.to_numpy(bquarks_from_higgs.pdgId, allow_missing=False)
nevents = b_quark_idx.shape[0]
firstgenpart_idxG = ak.firsts(genpart[:, 0].children).genPartIdxMotherG
firstgenpart_idxG_numpy = ak.to_numpy(firstgenpart_idxG, allow_missing=False)

In [38]:
@njit
def get_quark_last_copy(
    b_quarks_idx,
    b_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    genpart_pt_flat,
    nevents,
    firstgenpart_idxG_numpy,
):
    prints = True
    # print input array
    if prints:
        print("b_quarks_idx", b_quarks_idx)
        print("b_quarks_pdgId", b_quarks_pdgId)
        print("children_idxG_flat", children_idxG_flat)
        print("genpart_pdgId_flat", genpart_pdgId_flat)
        print("genpart_offsets", genpart_offsets)
        print("genpart_LastCopy_flat", genpart_LastCopy_flat)
        print("nevents", nevents)

    # get the children ofthe b_quarks which have the same pdgId of the mother iteratively until we reach the last copy

    out = np.zeros(b_quarks_idx.shape, dtype="int64") - 1

    for iev in range(b_quarks_idx.shape[0]):

        prints=True if iev==700 else False
        if prints:
            print("\nEvent", iev)
        for ipart in range(b_quarks_idx.shape[1]):
            p_id = b_quarks_idx[iev][ipart]
            if prints:
                print("Parton", ipart, genpart_LastCopy_flat[p_id], len(children_idxG_flat[p_id]), p_id)
            i = 0
            if genpart_LastCopy_flat[p_id] or len(children_idxG_flat[p_id]) == 0:
                out[iev][ipart] = p_id
                if prints:
                    print("out", out[iev][ipart], out[iev][ipart]-genpart_offsets[iev])
                continue
            while (not genpart_LastCopy_flat[p_id]) and (not len(children_idxG_flat[p_id]) == 0) and i<5:
                i += 1
                children_idxs = reverse_index_array(
                    children_idxG_flat[p_id],
                    firstgenpart_idxG_numpy,
                    genpart_offsets,
                    nevents,
                )
                # children_idxs = children_idxG_flat[p_id]
                if prints:
                    print(children_idxs)

                # get the children with the same pdgId as the mother with highest pt
                max_pt = -1
                max_pt_idx = p_id

                # num_pdg_equal=0
                # for child_idx in children_idxs:
                #     if genpart_pdgId_flat[child_idx] == b_quarks_pdgId[iev][ipart]:
                #         num_pdg_equal+=1
                # if num_pdg_equal==1:
                #     p_id = children_idxs[0]
                #     break
                # if prints: print("\n\n###################\n\n")

                for child_idx in children_idxs:
                    if prints:
                        print("Child", child_idx)
                    if prints:
                        print(b_quarks_pdgId[iev][ipart])
                    if prints:
                        print(genpart_pdgId_flat[child_idx])
                    if genpart_pdgId_flat[child_idx] != b_quarks_pdgId[iev][ipart]:
                        continue
                    child_pt = genpart_pt_flat[child_idx]
                    if prints:
                        print(child_pt)
                    if child_pt > max_pt:
                        max_pt_idx = child_idx
                        max_pt = child_pt

                if prints:
                    print("genpart_LastCopy_flat", genpart_LastCopy_flat[max_pt_idx])
                    print("len(children_idxG_flat[p_id])", len(children_idxG_flat[p_id]))
                if genpart_LastCopy_flat[max_pt_idx] or len(children_idxG_flat[max_pt_idx]) == 0 or max_pt==-1:
                    if prints:
                        print("Found child")
                    out[iev][ipart] = max_pt_idx
                if prints:
                    print(p_id, max_pt_idx)
                p_id = max_pt_idx
                if prints:
                    print(p_id, max_pt_idx)
                # break
                # if max_pt == -1:
                #     max_pt_idx = p_id
                if out[iev][ipart] != -1:
                    if prints:
                        print("break")
                    break
            if prints:
                print("out", out[iev][ipart], out[iev][ipart]-genpart_offsets[iev])

    return out

In [39]:
b_quark_last_idx = get_quark_last_copy(
    b_quark_idx,
    b_quarks_pdgId,
    children_idxG_flat,
    genpart_pdgId_flat,
    genpart_offsets,
    genpart_LastCopy_flat,
    genpart_pt_flat,
    nevents,
    firstgenpart_idxG_numpy,
)
print(b_quark_last_idx)
print(genpart_offsets[:-1])
print(ak.any(b_quark_last_idx==-1))


b_quarks_idx [[     17      18      19      20]
 [     87      88      89      90]
 [    154     155     156     157]
 ...
 [3692722 3692723 3692724 3692725]
 [3692786 3692787 3692788 3692789]
 [3692846 3692847 3692848 3692849]]
b_quarks_pdgId [[ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 ...
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]
 [ 5 -5  5 -5]]
children_idxG_flat [[2, 3, 4], [], [5], [6], [11, 12, 22, ... [], [3692885], [], [], [3692886], [], []]
genpart_pdgId_flat [21, 21, 25, 25, 21, 25, 25, 25, 25, ... -411, 411, 423, 14, -13, -423, 421, -421]
genpart_offsets [      0      71     137 ... 3692763 3692830 3692887]
genpart_LastCopy_flat [False, False, False, False, False, False, ... True, True, True, True, True, True]
nevents 57232

Event 700
Parton 0 False 9 45641
[45648 45650 45651 45655 45665 45667 45669 45686 45687]
Child 45648
5
21
Child 45650
5
5
13.84375
Child 45651
5
-523
Child 45655
5
-2
Child 45665
5
2
Child 45667
5
21
Child 45669
5
511
Child 45686
5
11
Child 45687
5
-11
genpart_Las

In [40]:
for i in range(len(b_quark_last_idx)):
    for j in range(len(b_quark_last_idx[i])):
        if b_quark_last_idx[i][j] == -1:
            print(b_quark_last_idx[i][j], b_quark_last_idx[i][j]-genpart_offsets[i], b_quarks_pdgId[i][j], genpart_pdgId_flat[b_quark_last_idx[i][j]])
            print(i, j)

In [ ]:
bq=genparts_flat[b_quark_last_idx]
print(bq.index, bq.pdgId, bq.genPartIdxMother, bq.pt)
print(bquarks_last.index, bquarks_last.pdgId, bquarks_last.genPartIdxMother, bquarks_last.pt, bquarks_last.hasFlags(["fromHardProcess"]))

In [ ]:
print("same index", ak.all(ak.sum(bq.index, axis=1)==ak.sum(bquarks_last.index, axis=1)))
bq_sum=ak.sum(bq.index, axis=1)
bquarks_last_sum=ak.sum(bquarks_last.index, axis=1)
num_gen_part=ak.num(genpart, axis=1)

In [ ]:
for i in range (len(bq_sum)):
    print(bq_sum[i], bquarks_last_sum[i],bq.index[i], bquarks_last.index[i], num_gen_part[i],genpart_offsets[i])

In [ ]:
genp=genparts_flat[218]
print(genp.index, genp.pdgId, genp.genPartIdxMother, genp.pt, genp.childrenIdxG, genp.children, genp.hasFlags(["isLastCopy"]), genp.hasFlags(["fromHardProcess"]))

In [ ]:
gen_old=genparts_flat[225]
print(gen_old.index, gen_old.pdgId, gen_old.genPartIdxMother, gen_old.pt, gen_old.childrenIdxG, gen_old.children, gen_old.hasFlags(["isLastCopy"]), gen_old.hasFlags(["fromHardProcess"]))

In [ ]:
# higgs_children=genparts_flat[higgs.childrenIdxG]
print(higgs.childrenIdxG)
print(ak.flatten(higgs.childrenIdxG))
higgs_children_flat2=ak.flatten(higgs.childrenIdxG, axis=None)
print(higgs_children_flat2)
print(genparts_flat)
higgs_children=genparts_flat[higgs_children_flat2]
print(higgs_children.index, higgs_children.pdgId, higgs_children.genPartIdxMother, higgs_children.pt, higgs_children.childrenIdxG, higgs_children.children, higgs_children.hasFlags(["isLastCopy"]), higgs_children.hasFlags(["fromHardProcess"]))

In [ ]:
print(b_quarks.index, b_quarks.pdgId, b_quarks.genPartIdxMother, b_quarks.pt)
print(bquarks_from_higgs.index, bquarks_from_higgs.pdgId, bquarks_from_higgs.genPartIdxMother, bquarks_from_higgs.pt)


In [ ]:
bquarks_from_higgs.statusFlags

In [ ]:
print(genpart.childrenIdxG)
print(~genpart.hasFlags(["isLastCopy"]), genpart.hasFlags(["isLastCopy"]))
condition=(~genpart.hasFlags(["isLastCopy"])) & ak.num(genpart.childrenIdxG, axis=2)==0
print(condition)
genpart_condition=genpart[condition]
print(genpart_condition.index, genpart_condition.pdgId, genpart_condition.genPartIdxMother, genpart_condition.pt, genpart_condition.childrenIdxG, genpart_condition.children, genpart_condition.hasFlags(["isLastCopy"]), genpart_condition.hasFlags(["fromHardProcess"]))